<a href="https://colab.research.google.com/github/connor-smith2/gym/blob/master/K_Cluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, LabelBinarizer
from sklearn.metrics import silhouette_samples
from sklearn.decomposition import PCA
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import seaborn as sns
import time

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

KeyboardInterrupt: ignored

In [ ]:
path = '/content/drive/My Drive/Data Testing Whole/input_data/training_data.csv'
data = pd.read_csv(path)
data.columns = ['frame_len', 'ip_src','ip_dst','tcp_src', 'tcp_dst','tcp_time', 'data_len']
origdata = data.copy()
#data.columns = ['frame_len', 'ip_src', 'tcp_time', 'data_len']
ip_src_max = data[["ip_src"]].max()
ip_dst_max = data[['ip_dst']].max()
tcp_src_max = data[['tcp_src']].max()
tcp_dst_max = data[['tcp_dst']].max()
data[["ip_src"]] /= ip_src_max
data[['ip_dst']] /= ip_dst_max
data[['tcp_src']] /= tcp_src_max
data[['tcp_dst']] /= tcp_dst_max
#print(data.head()) 
#scalar = MinMaxScaler(feature_range = (0,1))
#data[['ip_src', 'ip_dst']] = scalar.fit_transform(data[['ip_src', 'ip_dst']])
#data_scaled = pd.DataFrame(scalar.fit_transform(data), columns = data.columns)
TrainingData = data
TrainingData

In [ ]:
km = KMeans(n_clusters=9, init='k-means++', n_init=10, max_iter=300, tol=1e-04, random_state=0)

y_km=km.fit_predict(TrainingData)
km.cluster_centers_

In [ ]:
TrainingData.loc[0][0]
len(TrainingData)

In [ ]:
km.labels_[1:10000]
plt.plot(range(1,len(TrainingData)+1), km.labels_)

In [ ]:
plt.plot(range(1,100), km.labels_[1:100])

In [ ]:
x = np.linspace(0, 449088, 449088)
y = km.labels_

plt.plot(x, y, 'o', color='black');

In [ ]:
distortions = []
for i in range(1,10):
	km = KMeans(n_clusters=i, init='k-means++', n_init=10, max_iter=300, random_state=0)
	km.fit(TrainingData)
	distortions.append(km.inertia_)
plt.plot(range(1,10), distortions, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Distortion')
plt.show()

In [ ]:
distortion = ((TrainingData-km.cluster_centers_[y_km])**2.0).sum(axis = 1)
clusters = km.labels_
km.cluster_centers_[y_km].shape

In [ ]:
TrainingData['Clusters'] = clusters
TrainingData['Distortion'] = distortion
TrainingData['orig_ip'] = origdata['ip_src']

anomaly = list()
for i in range(0,len(distortion)):
  anomaly.append(1 if distortion[i] > 0.0003763 else 0)
TrainingData['Anomaly'] = anomaly
TrainingData
print(len(anomaly))
i = 0
for j in anomaly:
  if j == 1: i+=1
print(i)

In [ ]:
path2 = '/content/drive/My Drive/Data Testing Whole/input_data/test_data.csv'
testdata = pd.read_csv(path2)
normalizeStart = time.time()
testdata.columns = ['frame_len', 'ip_src','ip_dst','tcp_src', 'tcp_dst','tcp_time', 'data_len']
origtestdata = testdata.copy()
#testdata.columns = ['frame_len', 'ip_src', 'tcp_time', 'data_len']
testdata[["ip_src"]] /= ip_src_max
testdata[['ip_dst']] /= ip_dst_max
testdata[['tcp_src']] /= tcp_src_max
testdata[['tcp_dst']] /= tcp_dst_max
TestData = testdata
normalizeEnd = time.time()
print("Normalization Time: {}".format(normalizeEnd-normalizeStart))
start = time.time()
TestData.head(1)

In [ ]:
ytest_km = km.fit_predict(TestData)
end = time.time()
print("Prediction Time {}".format(end-start))

In [ ]:
TestData.shape
#x = ((TestData-km.cluster_centers_[ytest_km])**2.0).sum(axis = 1)
km.cluster_centers_[ytest_km].shape

In [ ]:
startDistortion = time.time()
distortion2 = ((TestData-km.cluster_centers_[ytest_km])**2.0).sum(axis = 1)
endDistortion = time.time()
print("Distortion Calc: {}".format(endDistortion-startDistortion))
#print(distortion2.sum())
anomaly2 = list()

startApp = time.time()
for i in range(0,len(distortion2)):
  anomaly2.append(1 if distortion2[i] > 0.0003763 else 0)
endApp = time.time()
print("Anomaly Detection Time: {}".format(endApp-startApp))
i = 0
for i in anomaly2:
  if i == 1: i+=1
print(i)

In [ ]:
TestData['Clusters'] = ytest_km
TestData['Distortion'] = distortion2
TestData['orig_ip'] = origtestdata['ip_src']
TestData['Anomaly'] = anomaly2
TestData

In [ ]:
print("Summary of Data:")
print("Training Data Head: ")
print(TrainingData.head())
print("Testing Data Head: ")
print(TestData.head())
print("Training Data Clusters: ", clusters)
print("Testing Data Clusters: ", ytest_km)
print("Training Data Distortion Sum: ", km.inertia_)
print("Testing Data Distortion Sum: ", distortion2.sum())
print("Training Data average Distortion :", TrainingData["Distortion"].mean())
print("Testing Data average Distortion :", TestData["Distortion"].mean())
print("Training Data Min Distortion :", TrainingData["Distortion"].min())
print("Testing Data Min Distortion :", TestData["Distortion"].min())
print("Training Data Max Distortion :", TrainingData["Distortion"].max())
print("Testing Data Max Distortion :", TestData["Distortion"].max())
#For testing TCP Time Interval:
print("Number of Anomalies :", TestData[TestData["Distortion"] >.0003763].count())